In [1]:
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import librosa

In [ ]:
df = pd.read_excel("filtered.xlsx").drop(columns=["file_name", "split_type"])
df

In [ ]:
df.district.unique()

In [ ]:
districts = ['Barishal', 'Chittagong', 'Comilla', 'Habiganj', 'Jhenaidah',
       'Kishoreganj', 'Lakshmipur', 'Narail', 'Narsingdi', 'Nilphamari',
       'Noakhali', 'Rangpur', 'Sandwip', 'Sylhet', 'Tangail']

def split_dataset(df, district):
    district = district.lower()
    total = df.shape[0]
    test = round(0.1*total)
    train = total - 2*test # both test and validation is 10% of the data, so 20%

    split_type = [[f"train_{district}_{i+1:04}.wav", "train"] for i in range(train)] \
                + [[f"test_{district}_{i+1:04}.wav", "test"] for i in range(test)] \
                + [[f"valid_{district}_{i+1:04}.wav", "valid"] for i in range(test)]

    df[["file_name", "split_type"]] = split_type
    return df[["file_name", "External_ID", "transcripts", "district", "split_type"]]
    
split_df = pd.DataFrame()

for district in districts:
    df2 = df.query("district == @district").copy()

    print(district)
    split_df = pd.concat([
        split_df, split_dataset(df2, district)
    ])
split_df

# Export junction table

In [11]:
split_df[["file_name", "External_ID", "district"]].to_excel("junction_table.xlsx", index=False)

# Export Dev version of split

In [10]:
split_df.to_excel("dev.xlsx", index=False)
split_df.query("split_type == 'train'").to_excel("dev_train.xlsx", index=False)
split_df.query("split_type == 'test'").to_excel("dev_test.xlsx", index=False)
split_df.query("split_type == 'valid'").to_excel("dev_valid.xlsx", index=False)

In [4]:
districts = ['Barishal', 'Chittagong', 'Comilla', 'Habiganj', 'Jhenaidah',
       'Kishoreganj', 'Lakshmipur', 'Narail', 'Narsingdi', 'Nilphamari',
       'Noakhali', 'Rangpur', 'Sandwip', 'Sylhet', 'Tangail']
df = pd.read_excel("dev\\dev.xlsx")

for district in districts:
    for sp in ["train", "test", "valid"]:
        df.query("district == @district and split_type == @sp").to_excel(f"district_wise\\{district}\\{district}_{sp}.xlsx", index=False)

In [5]:
for sp in ["train", "test", "valid"]:
    df.query("split_type == @sp")[["file_name", "transcripts"]].to_excel(f"{sp}.xlsx", index=False)